In [1]:
# %% [markdown]
# GeneralStudyAI: Agentic Multi-Subject Study Planner
# Kaggle Notebook Template
# Track: Concierge Agents
# Author: Generated by ChatGPT (GeneralStudyAI template)
#
# INSTRUCTIONS:
# - Beginner-friendly notebook that simulates agent behavior using simple Python classes
# - Replace stubs with real ADK/Vertex tools later if desired
# - No API keys required
#
# Sections:
# 1) Install & Imports
# 2) Agent Framework
# 3) Tools & Memory
# 4) Multi-Agent Flow Example
# 5) Logging / Observability
# 6) README + Artifacts
# 7) Video Script

# %%
# 1) INSTALL & IMPORTS
import json
import datetime
from typing import List, Dict, Any, Optional
import random
import textwrap
import os

# %% [markdown]
# 2) SIMPLE AGENT FRAMEWORK (SIMULATED)

# %%
class MemoryBank:
    """Simple long-term memory persisted to disk (JSON file)."""
    def __init__(self, path: str = '/kaggle/working/memory.json'):
        self.path = path
        self._data = self._load()

    def _load(self):
        try:
            with open(self.path, 'r') as f:
                return json.load(f)
        except Exception:
            return {'progress': [], 'user': {}}

    def save(self):
        os.makedirs(os.path.dirname(self.path), exist_ok=True)
        with open(self.path, 'w') as f:
            json.dump(self._data, f, indent=2)

    def append_progress(self, entry: Dict[str, Any]):
        entry['timestamp'] = datetime.datetime.utcnow().isoformat()
        self._data['progress'].append(entry)
        self.save()

    def get_progress(self):
        return self._data['progress']

    def set_user(self, user_info: Dict[str, Any]):
        self._data['user'] = user_info
        self.save()

    def get_user(self):
        return self._data.get('user', {})


class Tool:
    """Base tool class. Extend run()."""
    def run(self, query: str) -> str:
        raise NotImplementedError


class SearchTool(Tool):
    """Simulated resource search tool."""
    def run(self, query: str) -> str:
        sample = (
            f"Resources for '{query}':\n"
            "1) Short video tutorial link\n"
            "2) Practice exercises / question set\n"
            "3) Quick notes / summary sheet\n"
        )
        return sample


class Agent:
    """Base agent class."""
    def __init__(self, name: str, tools: Optional[List[Tool]] = None, memory: Optional[MemoryBank] = None):
        self.name = name
        self.tools = tools or []
        self.memory = memory

    def act(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        raise NotImplementedError


# %% [markdown]
# 3) DEFINE AGENTS (General Study Planner)

# %%
class PlanningAgent(Agent):
    def act(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        subjects = input_data.get('subjects', ['Math', 'Physics', 'Computer Science'])
        days_available = input_data.get('days_available', 7)
        daily_hours = input_data.get('daily_hours', 1)
        level = input_data.get('level', 'beginner')

        schedule = []

        for i in range(days_available):
            subj = subjects[i % len(subjects)]
            schedule.append({
                'day': i + 1,
                'subject': subj,
                'hours': daily_hours,
                'task': f"Study {subj}: read notes + practice problems ({daily_hours}h)"
            })

        output = {
            'planner': schedule,
            'notes': f"Generated a {days_available}-day plan across {len(subjects)} subjects."
        }

        if self.memory:
            self.memory.append_progress({
                'agent': self.name,
                'action': 'generated_plan',
                'summary': output['notes']
            })

        return output


class TaskAgent(Agent):
    def act(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        subject = input_data.get('subject', 'Math')
        search_tool = next((t for t in self.tools if isinstance(t, SearchTool)), None)
        resources = search_tool.run(subject) if search_tool else 'Search tool unavailable.'

        task = f"Complete 10 practice problems or 1 chapter revision in {subject}."

        output = {
            'subject': subject,
            'resources': resources,
            'task': task
        }

        if self.memory:
            self.memory.append_progress({
                'agent': self.name,
                'action': 'suggested_task',
                'subject': subject
            })

        return output


class DoubtAgent(Agent):
    def act(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        question = input_data.get('question', '')
        answer = f"(Simulated answer) For '{question}', break it into smaller steps, apply formulas, and review examples."
        output = {'question': question, 'answer': answer}

        if self.memory:
            self.memory.append_progress({
                'agent': self.name,
                'action': 'answered_question',
                'question': question
            })

        return output


# %% [markdown]
# 4) SIMPLE LOGGER

# %%
class SimpleLogger:
    def __init__(self):
        self.logs = []

    def info(self, msg: str):
        entry = {
            'level': 'INFO',
            'time': datetime.datetime.utcnow().isoformat(),
            'msg': msg
        }
        self.logs.append(entry)
        print('[INFO]', msg)

logger = SimpleLogger()

# %% [markdown]
# 5) RUN MULTI-AGENT EXAMPLE

# %%
memory = MemoryBank(path='/kaggle/working/memory.json')
search_tool = SearchTool()

planning_agent = PlanningAgent('PlanningAgent', tools=[search_tool], memory=memory)
task_agent = TaskAgent('TaskAgent', tools=[search_tool], memory=memory)
doubt_agent = DoubtAgent('DoubtAgent', tools=[search_tool], memory=memory)

user_profile = {
    'name': 'Student',
    'subjects': ['Math', 'Physics', 'Computer Science', 'English'],
    'days_available': 7,
    'daily_hours': 1,
    'level': 'beginner'
}

memory.set_user(user_profile)

logger.info("Running PlanningAgent...")
plan_output = planning_agent.act(user_profile)
print(json.dumps(plan_output, indent=2))

first_subject = plan_output['planner'][0]['subject']
logger.info(f"Running TaskAgent for subject: {first_subject}")
task_output = task_agent.act({'subject': first_subject})
print(json.dumps(task_output, indent=2))

logger.info("Running DoubtAgent...")
doubt_output = doubt_agent.act({
    'question': 'How do I study integration effectively?'
})
print(json.dumps(doubt_output, indent=2))


# %% [markdown]
# 6) GENERATE README

# %%
README = textwrap.dedent(f"""
# GeneralStudyAI: Agentic Multi-Subject Study Planner

## Summary
GeneralStudyAI is a beginner-friendly agent-based study planner that creates personalized study schedules,
suggests tasks, and answers basic study doubts across multiple subjects.

## How to run
1. Open this notebook in Kaggle or Colab.
2. Run all cells.
3. Agents will generate plans, tasks, and explanations.

## Files
- `GeneralStudyAI_Notebook.ipynb`
- `/kaggle/working/memory.json` (memory)

""")

with open('/kaggle/working/README.md', 'w') as f:
    f.write(README)

print("README saved → /kaggle/working/README.md")

# %% [markdown]
# 7) VIDEO SCRIPT

# %%
VIDEO_SCRIPT = textwrap.dedent("""
0:00 - 0:10 Introduction: "Hi, I'm <Your Name>, and this is GeneralStudyAI."
0:10 - 0:30 Problem: Students struggle to organize multi-subject study plans.
0:30 - 0:50 Why Agents: Agents automate planning and task suggestions.
0:50 - 1:30 Architecture: Planner → TaskAgent → DoubtAgent → Memory.
1:30 - 2:00 Demo: Show sample plan and doubt answer.
2:00 - 2:40 Build: Explain agent classes, tools, and memory.
2:40 - 3:00 Conclusion: Where to find code and notebook.
""")

with open('/kaggle/working/VIDEO_SCRIPT.txt', 'w') as f:
    f.write(VIDEO_SCRIPT)

print("Video script saved → /kaggle/working/VIDEO_SCRIPT.txt")

print("\nAll steps complete. You can now publish your notebook.")


[INFO] Running PlanningAgent...
{
  "planner": [
    {
      "day": 1,
      "subject": "Math",
      "hours": 1,
      "task": "Study Math: read notes + practice problems (1h)"
    },
    {
      "day": 2,
      "subject": "Physics",
      "hours": 1,
      "task": "Study Physics: read notes + practice problems (1h)"
    },
    {
      "day": 3,
      "subject": "Computer Science",
      "hours": 1,
      "task": "Study Computer Science: read notes + practice problems (1h)"
    },
    {
      "day": 4,
      "subject": "English",
      "hours": 1,
      "task": "Study English: read notes + practice problems (1h)"
    },
    {
      "day": 5,
      "subject": "Math",
      "hours": 1,
      "task": "Study Math: read notes + practice problems (1h)"
    },
    {
      "day": 6,
      "subject": "Physics",
      "hours": 1,
      "task": "Study Physics: read notes + practice problems (1h)"
    },
    {
      "day": 7,
      "subject": "Computer Science",
      "hours": 1,
      "task": "S